In [2]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Cargar los datos

In [ ]:
ruta_al_archivo = 'datos_clientes.csv'  # Modifica según sea necesario
data = pd.read_csv(ruta_al_archivo)

# 2. Preprocesamiento de datos

In [ ]:
# Manejo de valores faltantes
data.fillna(data.median(numeric_only=True), inplace=True)
data.fillna('Desconocido', inplace=True)

# Codificación de variables categóricas
label_encoders = {}
for col in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Normalización de variables numéricas
scaler = StandardScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(data.select_dtypes(include=['number'])), 
                           columns=data.select_dtypes(include=['number']).columns)

# 3. Selección de características

In [ ]:
# Para simplificar, usamos todas las columnas numéricas
X = data_scaled

# 4. Determinación del número óptimo de clusters

In [ ]:
wcss = []
silhouette_scores = []

for i in range(2, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X, kmeans.labels_))

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(2, 11), wcss, marker='o')
plt.title('Método del codo')
plt.xlabel('Número de clusters')
plt.ylabel('WCSS')
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(range(2, 11), silhouette_scores, marker='o')
plt.title('Coeficiente de silueta')
plt.xlabel('Número de clusters')
plt.ylabel('Puntuación de silueta')
plt.show()

In [ ]:
n_clusters_optimo = 4  # Cambia según los gráficos

# 5. Entrenamiento del modelo K-Means

In [ ]:
kmeans = KMeans(n_clusters=n_clusters_optimo, random_state=42)
data['Cluster'] = kmeans.fit_predict(X)

# 6. Verificación del modelo

In [ ]:
# Coeficiente de silueta
silhouette_avg = silhouette_score(X, data['Cluster'])
print(f'Coeficiente de silueta promedio: {silhouette_avg:.2f}')

In [ ]:
# Índice Calinski-Harabasz
calinski_score = calinski_harabasz_score(X, data['Cluster'])
print(f'Índice de Calinski-Harabasz: {calinski_score:.2f}')

In [ ]:
# Índice Davies-Bouldin (valores más bajos son mejores)
davies_bouldin = davies_bouldin_score(X, data['Cluster'])
print(f'Índice Davies-Bouldin: {davies_bouldin:.2f}')

In [ ]:
# Verificar distribución de clusters
cluster_counts = data['Cluster'].value_counts()
print("\nDistribución de puntos por cluster:")
print(cluster_counts)

In [ ]:
plt.figure(figsize=(8, 5))
sns.barplot(x=cluster_counts.index, y=cluster_counts.values)
plt.title('Distribución de puntos por cluster')
plt.xlabel('Cluster')
plt.ylabel('Número de puntos')
plt.show()

In [ ]:
# Validación visual con PCA para reducir dimensiones
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
data['PCA1'], data['PCA2'] = X_pca[:, 0], X_pca[:, 1]

In [ ]:
plt.figure(figsize=(10, 8))
sns.scatterplot(data=data, x='PCA1', y='PCA2', hue='Cluster', palette='Set1', alpha=0.7)
plt.title('Visualización de clusters (PCA)')
plt.xlabel('Componente principal 1')
plt.ylabel('Componente principal 2')
plt.legend(title='Cluster')
plt.show()

In [ ]:
# Guardar datos segmentados
data.to_csv('clientes_segmentados_validado.csv', index=False)
print("Los datos segmentados y validados se han guardado en 'clientes_segmentados_validado.csv'.")